In [1]:
# imports

import os
# import urllib2
import pandas as pd
import urllib.request, json
# from pprint import pprint
from pandas.io.json import json_normalize
# import matplotlib.pyplot as plt
import datetime

In [2]:
# load api from covidtracking in json form

with urllib.request.urlopen("https://covidtracking.com/api/v1/states/daily.json") as url:
    states_historical = json.loads(url.read().decode())
#     pprint(states_historical)

In [3]:
# jsonify as pandas df

state_historical_df = json_normalize(states_historical[0:]).fillna(0)
state_historical_df

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200505,AK,371.0,22321.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,22692,22692,22692,02,0.0,0.0,968.0,1.0,969.0
1,20200505,AL,8285.0,98481.0,0.0,0.0,1107.0,0.0,428.0,0.0,...,1107.0,106766,106766,106766,01,17.0,43.0,3389.0,260.0,3649.0
2,20200505,AR,3496.0,51139.0,0.0,89.0,453.0,0.0,0.0,16.0,...,453.0,54635,54635,54635,05,-1.0,15.0,155.0,38.0,193.0
3,20200505,AS,0.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,83,83,83,60,0.0,0.0,0.0,0.0,0.0
4,20200505,AZ,9305.0,78955.0,0.0,728.0,1397.0,303.0,0.0,185.0,...,1397.0,88260,88260,88260,04,33.0,40.0,2621.0,386.0,3007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,20200126,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,1,53,0.0,0.0,0.0,0.0,0.0
3429,20200125,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,1,53,0.0,0.0,0.0,0.0,0.0
3430,20200124,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,1,53,0.0,0.0,0.0,0.0,0.0
3431,20200123,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,1,53,0.0,0.0,0.0,0.0,0.0


In [4]:
# load csv file with state populations and calculate population per millions

popul = os.path.join('./Resources/population.csv')
popul_df = pd.read_csv(popul)
popul_df.columns = ['state', 'population', 'pop (mil)']
# popul_df.dtypes
popul_df.head()

,state,population,pop (mil)
0,AL,"4,903,185",4.90
1,AK,"731,545",0.73
2,AZ,"7,278,717",7.28
3,AR,"3,017,804",3.02
4,CA,"39,512,223",39.51


In [5]:
# merge two df

conct_df = pd.merge(state_historical_df,
                 popul_df[['state', 'population', 'pop (mil)']], on = 'state')

# conct_df.head()

In [6]:
# sort by state (leaving dates until after daily avgs are calculated)

sorted_hist_df = conct_df.sort_values(by=['state', 'date'], ascending=[True, True])
sorted_hist_df = sorted_hist_df.reset_index(drop=True)
# sorted_hist_df

In [7]:
# sort new df by columns, rename columns, convert date and reindex

# sorted_df = state_historical_df.loc[state_historical_df['state'] == state]
sorted_hist_df = sorted_hist_df[['date', 'state', 'population', 'pop (mil)', 'positive', 'positiveIncrease','negative', 'negativeIncrease', 
                     'totalTestResults', 'totalTestResultsIncrease', 'hospitalized', 'hospitalizedIncrease',
                     'death', 'deathIncrease']].fillna(0)

# sorted_hist_df = sorted_hist_df.iloc[::-1]
sorted_hist_df['date'] = pd.to_datetime(sorted_hist_df['date'].astype(str), format='%Y%m%d')
sorted_hist_df = sorted_hist_df.reset_index(drop=True)

sorted_hist_df.rename(columns = {'date':'Date', 'state':'State', 'population':'Population', 'pop (mil)':'Pop (mil)', 'positive':'Total Positive Cases', 'positiveIncrease':'Positives Increase',
                           'negative':'Negative Cases', 'negativeIncrease':'Negatives Increase', 'totalTestResults':'Total Tests',
                           'totalTestResultsIncrease':'Total Tests Increase', 'hospitalized':'Total Hospitalized',
                           'hospitalizedIncrease':'Hospitalized Increase', 'death':'Total Deaths', 'deathIncrease':'Deaths Increase'}, inplace = True)
# state_df.drop(['date'])

# sorted_hist_df

In [8]:
# Calculate and add 3 day positives increase avg using for loop

sorted_hist_df.insert(loc=6, column='3 Day Pos Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Pos Inc Avg'] = round((sorted_hist_df.loc[i, 'Positives Increase'] + 
    sorted_hist_df.loc[i-1, 'Positives Increase'] + 
    sorted_hist_df.loc[i-2, 'Positives Increase'])/3,2)
    
# zero out first 2 days of 3 day increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):
    
    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Pos Inc Avg'] = 0
    
sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,8.0,0.0,8,0.0,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,12.0,4.0,12,4.0,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,14.0,2.0,14,2.0,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,23.0,9.0,23,9.0,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,23.0,0.0,23,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2020-05-01,WY,"578,759",0.58,559.0,15.0,56.67,9136.0,234.0,9695,249.0,56.0,0.0,7.0,0.0
3174,2020-05-02,WY,"578,759",0.58,566.0,7.0,10,9463.0,327.0,10029,334.0,58.0,2.0,7.0,0.0
3175,2020-05-03,WY,"578,759",0.58,579.0,13.0,11.67,9640.0,177.0,10219,190.0,60.0,2.0,7.0,0.0
3176,2020-05-04,WY,"578,759",0.58,586.0,7.0,9,9868.0,228.0,10454,235.0,60.0,0.0,7.0,0.0


In [9]:
# Calculate and add 7 day positives increase avg using for loop


sorted_hist_df.insert(loc=7, column='7 Day Pos Inc Avg', value ='')

for i in range(7, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '7 Day Pos Inc Avg'] = round((sorted_hist_df.loc[i, 'Positives Increase'] + 
    sorted_hist_df.loc[i-1, 'Positives Increase'] + 
    sorted_hist_df.loc[i-2, 'Positives Increase'] +
    sorted_hist_df.loc[i-3, 'Positives Increase'] + 
    sorted_hist_df.loc[i-4, 'Positives Increase'] +
    sorted_hist_df.loc[i-5, 'Positives Increase'] + 
    sorted_hist_df.loc[i-6, 'Positives Increase'])/7,2)
    
# zero out first 7 days of 7 day increase avg using for loop - new state was calculating off of previous states last days
for i in range(7, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+2, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+3, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+4, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+5, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+6, '7 Day Pos Inc Avg'] = 0

sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8,0.0,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12,4.0,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14,2.0,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23,9.0,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2020-05-01,WY,"578,759",0.58,559.0,15.0,56.67,30,9136.0,234.0,9695,249.0,56.0,0.0,7.0,0.0
3174,2020-05-02,WY,"578,759",0.58,566.0,7.0,10,31,9463.0,327.0,10029,334.0,58.0,2.0,7.0,0.0
3175,2020-05-03,WY,"578,759",0.58,579.0,13.0,11.67,31,9640.0,177.0,10219,190.0,60.0,2.0,7.0,0.0
3176,2020-05-04,WY,"578,759",0.58,586.0,7.0,9,30.86,9868.0,228.0,10454,235.0,60.0,0.0,7.0,0.0


In [10]:
sorted_hist_df[265:280
            ]

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
265,2020-03-24,CA,"39,512,223",39.51,2102.0,369.0,274.33,231.29,13452.0,885.0,15554,1254.0,0.0,0.0,40.0,13.0
266,2020-03-25,CA,"39,512,223",39.51,2355.0,253.0,273,249.14,15921.0,2469.0,18276,2722.0,0.0,0.0,53.0,13.0
267,2020-03-26,CA,"39,512,223",39.51,3006.0,651.0,424.33,297.43,17380.0,1459.0,20386,2110.0,0.0,0.0,65.0,12.0
268,2020-03-27,CA,"39,512,223",39.51,3879.0,873.0,592.33,402.29,17380.0,0.0,21259,873.0,0.0,0.0,78.0,13.0
269,2020-03-28,CA,"39,512,223",39.51,4643.0,764.0,762.67,480.57,20549.0,3169.0,25192,3933.0,0.0,0.0,101.0,23.0
270,2020-03-29,CA,"39,512,223",39.51,5708.0,1065.0,900.67,596,20549.0,0.0,26257,1065.0,0.0,0.0,123.0,22.0
271,2020-03-30,CA,"39,512,223",39.51,6447.0,739.0,856,673.43,20549.0,0.0,26996,739.0,0.0,0.0,133.0,10.0
272,2020-03-31,CA,"39,512,223",39.51,7482.0,1035.0,946.33,768.57,21772.0,1223.0,29254,2258.0,0.0,0.0,153.0,20.0
273,2020-04-01,CA,"39,512,223",39.51,8155.0,673.0,815.67,828.57,21772.0,0.0,29927,673.0,0.0,0.0,171.0,18.0
274,2020-04-02,CA,"39,512,223",39.51,9191.0,1036.0,914.67,883.57,23809.0,2037.0,33000,3073.0,0.0,0.0,203.0,32.0


In [11]:
sorted_hist_df.columns

Index(['Date', 'State', 'Population', 'Pop (mil)', 'Total Positive Cases',
       'Positives Increase', '3 Day Pos Inc Avg', '7 Day Pos Inc Avg',
       'Negative Cases', 'Negatives Increase', 'Total Tests',
       'Total Tests Increase', 'Total Hospitalized', 'Hospitalized Increase',
       'Total Deaths', 'Deaths Increase'],
      dtype='object')

In [12]:
# Calculate and add 3 day total testing avg using for loop

sorted_hist_df.insert(loc=12, column='3 Day Testing Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Testing Inc Avg'] = round((sorted_hist_df.loc[i, 'Total Tests Increase'] + 
    sorted_hist_df.loc[i-1, 'Total Tests Increase'] + 
    sorted_hist_df.loc[i-2, 'Total Tests Increase'])/3,2)
    
# zero out first 2 days of 3 day testing increase avg using for loop - new state was calculating off of previous states last days    
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Testing Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Testing Inc Avg'] = 0
        
sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8,0.0,,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12,4.0,,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14,2.0,2,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23,9.0,5,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23,0.0,3.67,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2020-05-01,WY,"578,759",0.58,559.0,15.0,56.67,30,9136.0,234.0,9695,249.0,360,56.0,0.0,7.0,0.0
3174,2020-05-02,WY,"578,759",0.58,566.0,7.0,10,31,9463.0,327.0,10029,334.0,270.67,58.0,2.0,7.0,0.0
3175,2020-05-03,WY,"578,759",0.58,579.0,13.0,11.67,31,9640.0,177.0,10219,190.0,257.67,60.0,2.0,7.0,0.0
3176,2020-05-04,WY,"578,759",0.58,586.0,7.0,9,30.86,9868.0,228.0,10454,235.0,253,60.0,0.0,7.0,0.0


In [13]:
sorted_hist_df.columns

Index(['Date', 'State', 'Population', 'Pop (mil)', 'Total Positive Cases',
       'Positives Increase', '3 Day Pos Inc Avg', '7 Day Pos Inc Avg',
       'Negative Cases', 'Negatives Increase', 'Total Tests',
       'Total Tests Increase', '3 Day Testing Inc Avg', 'Total Hospitalized',
       'Hospitalized Increase', 'Total Deaths', 'Deaths Increase'],
      dtype='object')

In [14]:
# Calculate and add 3 day hospitalized inc avg using for loop


sorted_hist_df.insert(loc=15, column='3 Day Hospitalized Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Hospitalized Inc Avg'] = round((sorted_hist_df.loc[i, 'Hospitalized Increase'] + 
    sorted_hist_df.loc[i-1, 'Hospitalized Increase'] + 
    sorted_hist_df.loc[i-2, 'Hospitalized Increase'])/3,2)
    
# zero out first 2 days of 3 day hospitalized increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Hospitalized Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Hospitalized Inc Avg'] = 0

sorted_hist_df
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,3 Day Hospitalized Inc Avg,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8,0.0,,0.0,0.0,,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12,4.0,,0.0,0.0,,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14,2.0,2,0.0,0.0,0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23,9.0,5,0.0,0.0,0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23,0.0,3.67,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2020-05-01,WY,"578,759",0.58,559.0,15.0,56.67,30,9136.0,234.0,9695,249.0,360,56.0,0.0,0,7.0,0.0
3174,2020-05-02,WY,"578,759",0.58,566.0,7.0,10,31,9463.0,327.0,10029,334.0,270.67,58.0,2.0,0.67,7.0,0.0
3175,2020-05-03,WY,"578,759",0.58,579.0,13.0,11.67,31,9640.0,177.0,10219,190.0,257.67,60.0,2.0,1.33,7.0,0.0
3176,2020-05-04,WY,"578,759",0.58,586.0,7.0,9,30.86,9868.0,228.0,10454,235.0,253,60.0,0.0,1.33,7.0,0.0


In [15]:
# Calculate and add 3 day deaths inc avg using for loop


sorted_hist_df.insert(loc=18, column='3 Day Deaths Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Deaths Inc Avg'] = round((sorted_hist_df.loc[i, 'Deaths Increase'] + 
    sorted_hist_df.loc[i-1, 'Deaths Increase'] + 
    sorted_hist_df.loc[i-2, 'Deaths Increase'])/3,2)

# zero out first 2 days of 3 day deaths increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Deaths Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Deaths Inc Avg'] = 0
        
sorted_hist_df.fillna(0)
    


,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,3 Day Hospitalized Inc Avg,Total Deaths,Deaths Increase,3 Day Deaths Inc Avg
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8,0.0,,0.0,0.0,,0.0,0.0,
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12,4.0,,0.0,0.0,,0.0,0.0,
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14,2.0,2,0.0,0.0,0,0.0,0.0,0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23,9.0,5,0.0,0.0,0,0.0,0.0,0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23,0.0,3.67,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2020-05-01,WY,"578,759",0.58,559.0,15.0,56.67,30,9136.0,234.0,9695,249.0,360,56.0,0.0,0,7.0,0.0,0
3174,2020-05-02,WY,"578,759",0.58,566.0,7.0,10,31,9463.0,327.0,10029,334.0,270.67,58.0,2.0,0.67,7.0,0.0,0
3175,2020-05-03,WY,"578,759",0.58,579.0,13.0,11.67,31,9640.0,177.0,10219,190.0,257.67,60.0,2.0,1.33,7.0,0.0,0
3176,2020-05-04,WY,"578,759",0.58,586.0,7.0,9,30.86,9868.0,228.0,10454,235.0,253,60.0,0.0,1.33,7.0,0.0,0


In [16]:
# calculations

sorted_hist_df.insert(loc=8, column='Positive Cases per Mil', value ='')
sorted_hist_df.insert(loc=14, column='Tests per Mil', value ='')
sorted_hist_df.insert(loc=18, column='Hospitalized per Mil', value ='')
sorted_hist_df.insert(loc=22, column='Deaths per Mil', value ='')
sorted_hist_df.insert(loc=12, column='Testing %', value ='')
sorted_hist_df.insert(loc=5, column='Positive Cases %', value ='')
sorted_hist_df.insert(loc=17, column='Tests per Mil/Positive Cases', value ='')
sorted_hist_df.insert(loc=18, column='Tests per Mil/Deaths', value ='')
sorted_hist_df.insert(loc=27, column='Deaths/Total Tests', value ='')
sorted_hist_df.insert(loc=28, column='Deaths/Positive Cases', value ='')
sorted_hist_df.insert(loc=20, column='Hospitalized %', value ='')
sorted_hist_df.insert(loc=25, column='Death %', value ='')


sorted_hist_df['Positive Cases per Mil'] = round(sorted_hist_df['Total Positive Cases']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Tests per Mil'] = round(sorted_hist_df['Total Tests']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Hospitalized per Mil'] = round(sorted_hist_df['Total Hospitalized']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Deaths per Mil'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Testing %'] = round(sorted_hist_df['Total Tests']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Positive Cases %'] = round(sorted_hist_df['Total Positive Cases']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Tests per Mil/Positive Cases'] = round(sorted_hist_df['Tests per Mil']/sorted_hist_df['Total Positive Cases'],2)
sorted_hist_df['Tests per Mil/Deaths'] = round(sorted_hist_df['Tests per Mil']/sorted_hist_df['Total Deaths'],2)
sorted_hist_df['Deaths/Total Tests'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Total Tests']*100,2)
sorted_hist_df['Deaths/Positive Cases'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Total Positive Cases']*100,2)
sorted_hist_df['Hospitalized %'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Death %'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)']/10000,3)

sorted_hist_df.head()

,Date,State,Population,Pop (mil),Total Positive Cases,Positive Cases %,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Positive Cases per Mil,...,Hospitalized Increase,3 Day Hospitalized Inc Avg,Hospitalized per Mil,Total Deaths,Death %,Deaths Increase,3 Day Deaths Inc Avg,Deaths per Mil,Deaths/Total Tests,Deaths/Positive Cases
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,0.0,,,0.0,...,0.0,,0.0,0.0,0.0,0.0,,0.0,0.0,NaN
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,0.0,,,0.0,...,0.0,,0.0,0.0,0.0,0.0,,0.0,0.0,NaN
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN


In [17]:
# resort df by descending date by state after all calculations

sorted_hist_df = sorted_hist_df.sort_values(by=['State', 'Date'], ascending=[True, False])
sorted_hist_df = sorted_hist_df.reset_index(drop=True)

# sorted_hist_df

In [18]:
# create datestamp for saving file daily

now = datetime.datetime.now()
# print ("Current date and time : ")
Previous_Date = datetime.datetime.today() - datetime.timedelta(days=1)
today = now.strftime("%Y-%m-%d")
yesterday = Previous_Date.strftime("%Y-%m-%d")
print(yesterday)

2020-05-05


In [19]:
# use todays date and sort by state to make daily df

# print(df.loc[df['A'] == 'foo'])
# state_update_df = sorted_hist_df.loc[sorted_hist_df['Date'] == 'today']
state_update_df = sorted_hist_df.loc[sorted_hist_df['Date'] == yesterday]

states50_COVID19_df = state_update_df.reset_index(drop=True)

states50_COVID19_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positive Cases %,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Positive Cases per Mil,...,Hospitalized Increase,3 Day Hospitalized Inc Avg,Hospitalized per Mil,Total Deaths,Death %,Deaths Increase,3 Day Deaths Inc Avg,Deaths per Mil,Deaths/Total Tests,Deaths/Positive Cases
0,2020-05-05,AK,"731,545",0.73,371.0,0.051,1.0,2,2.86,508.22,...,0.0,0,0.00,9.0,0.001,0.0,0,12.33,0.04,2.43
1,2020-05-05,AL,"4,903,185",4.90,8285.0,0.169,260.0,283.67,228.29,1690.82,...,43.0,28,225.92,313.0,0.006,17.0,8.33,63.88,0.29,3.78
2,2020-05-05,AR,"3,017,804",3.02,3496.0,0.116,38.0,41.33,55,1157.62,...,15.0,13,150.00,80.0,0.003,-1.0,2.33,26.49,0.15,2.29
3,2020-05-05,AZ,"7,278,717",7.28,9305.0,0.128,386.0,313.67,336.71,1278.16,...,40.0,19.33,191.90,395.0,0.005,33.0,15.67,54.26,0.45,4.25
4,2020-05-05,CA,"39,512,223",39.51,56212.0,0.142,1275.0,1338.33,1597.29,1422.73,...,0.0,0,0.00,2317.0,0.006,63.0,48.67,58.64,0.30,4.12
5,2020-05-05,CO,"5,758,736",5.76,16907.0,0.294,272.0,379.67,432.57,2935.24,...,39.0,30.33,492.71,851.0,0.015,9.0,10.33,147.74,1.00,5.03
6,2020-05-05,CT,"3,565,287",3.57,30621.0,0.858,1334.0,619,615.57,8577.31,...,0.0,0,2173.11,2633.0,0.074,138.0,98,737.54,2.42,8.60
7,2020-05-05,DC,"705,749",0.71,5322.0,0.750,152.0,175,189.71,7495.77,...,0.0,0,0.00,264.0,0.037,6.0,8,371.83,1.09,4.96
8,2020-05-05,DE,"973,764",0.97,5371.0,0.554,83.0,111,113.71,5537.11,...,0.0,0,0.00,187.0,0.019,5.0,6.33,192.78,0.76,3.48
9,2020-05-05,FL,"21,477,737",21.48,37439.0,0.174,542.0,658.67,656.14,1742.97,...,217.0,200.33,304.75,1536.0,0.007,113.0,49.33,71.51,0.33,4.10


In [20]:
# # save to file - will import to tableau

sorted_hist_df.to_csv('./Resources/COVID19.csv')

MN_daily = sorted_hist_df.loc[sorted_hist_df['State'] == 'MN']
WI_daily = sorted_hist_df.loc[sorted_hist_df['State'] == 'WI']

MN_daily.to_csv('./Resources/MN_COVID19.csv')
WI_daily.to_csv('./Resources/WI_COVID19.csv')

states50_COVID19_df.to_csv('./Resources/50states_COVID19.csv')



In [21]:
# d1 = state_df['Deaths']
# d2 = state_df['Positive Cases']
# t = state_df['Date']

# plt.plot(t, d1, "-b", label="Deaths")
# plt.plot(t, d2, "-r", label="Positive Cases")
# plt.legend(loc="upper left")
# plt.title(state + ' Positive Covid-19 Cases and Deaths')
# # plt.set_xticklabels(t, rotation = 45)
# plt.xticks(rotation=45)

# plt.show()